# Spike Pattern Identifier

Interactive notebook for analyzing neural spike patterns with configurable parameters.
This notebook uses the custom `utils.py` module to identify different firing patterns in spike data.

## 1. Import Required Libraries and Utils Module

Import all necessary libraries and our custom pattern analysis utilities.

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display, clear_output
import json
import warnings
warnings.filterwarnings('ignore')

# Import our custom utils module
from utils import SpikePatternAnalyzer, load_spike_data, load_electrode_config, plot_unit_analysis, print_data_summary

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")

## 2. Load and Prepare Unit Data

Load spike data and electrode configuration, then initialize the pattern analyzer.

In [2]:
# Load data files
spike_data = load_spike_data('spikes.csv')
electrode_config = load_electrode_config('electrode.cfg')
analyzer = SpikePatternAnalyzer(spike_data, electrode_config)

# Print summary using utils function
print_data_summary(spike_data, electrode_config)

# Get basic info for widgets
unique_units = sorted(spike_data['unit'].unique())
unique_electrodes = sorted(spike_data['electrode'].unique())

print(f"\nFirst few units available: {unique_units[:10]}")

Dataset Summary:
  • Recording duration: 909.22 seconds
  • Number of units: 45
  • Number of electrodes: 20
  • Total spikes: 290,775
  • Electrode configuration loaded: 32 electrodes

First few units available: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10)]


## 3. Define Pattern Identification Parameters

Set up configurable parameters for different types of pattern analysis.

In [ ]:
# Create compact parameter widgets
unit_selector = widgets.Dropdown(
    options=unique_units,
    value=unique_units[0],
    description='Unit ID:'
)

burst_max_isi = widgets.FloatSlider(
    value=0.01, min=0.001, max=0.1, step=0.001,
    description='Burst Max ISI (s):', readout_format='.3f'
)

burst_min_spikes = widgets.IntSlider(
    value=3, min=2, max=10,
    description='Min Spikes in Burst:'
)

fr_window_size = widgets.FloatSlider(
    value=1.0, min=0.1, max=5.0, step=0.1,
    description='FR Window (s):'
)

# Analysis button
analyze_button = widgets.Button(
    description='Set Parameters',
    button_style='success'
)

# Compact parameter layout
parameter_box = widgets.VBox([
    widgets.HTML("<h3>Analysis Parameters</h3>"),
    unit_selector,
    burst_max_isi,
    burst_min_spikes,
    fr_window_size,
    analyze_button
])

display(parameter_box)
print("Parameter widgets ready!")

Parameter widgets ready!


## 4. Run Pattern Identifier for Specific Units

Execute pattern identification with the selected parameters and display results.

In [ ]:
# Run pattern analysis with selected parameters
def run_pattern_analysis():
    # Get parameters from widgets
    params = {
        'burst_max_isi': burst_max_isi.value,
        'burst_min_spikes': burst_min_spikes.value,
        'firing_rate_window': fr_window_size.value,
    }
    
    current_unit = unit_selector.value
    print(f"Running analysis for Unit {current_unit}...")
    
    # Run analysis
    results = analyzer.run_pattern_analysis(current_unit, params)
    
    # Print results using utils function
    analyzer.print_analysis_results(results, current_unit)
    
    return results, current_unit

# Run the analysis automatically
current_results, current_unit = run_pattern_analysis()

Running analysis for Unit 1...


## 5. Visualize Identified Patterns

Create comprehensive visualizations of the identified patterns.

In [ ]:
# Create visualizations for the analyzed unit
if current_results is None:
    print("No analysis results available. Please run the analysis cell above first.")
elif 'error' in current_results:
    print(f"Cannot visualize results due to error: {current_results['error']}")
else:
    print(f"Creating visualizations for Unit {current_unit}...")
    
    try:
        # Use the comprehensive plotting function from utils
        fig = plot_unit_analysis(analyzer, current_unit, current_results, figsize=(16, 12))
        plt.show()
        print("Visualization completed successfully!")
        
    except Exception as e:
        print(f"Error creating plots: {str(e)}")

Visualization function ready!


Button(button_style='info', description='Generate Visualizations', style=ButtonStyle(), tooltip='Create compre…

## 6. Export Pattern Results

Save the identified patterns to files for further analysis.

In [ ]:
# Simplified export functionality
import json

export_button = widgets.Button(
    description='Export Current Results',
    button_style='warning'
)

def export_results(button):
    global current_results, current_unit
    
    if current_results is None:
        print("No analysis results available. Please run pattern analysis first.")
        return
    
    if 'error' in current_results:
        print(f"Cannot export results due to error: {current_results['error']}")
        return
    
    filename = f"pattern_analysis_unit_{current_unit}.json"
    
    try:
        # Simple export - convert numpy arrays to lists
        export_data = {}
        for key, value in current_results.items():
            if hasattr(value, 'tolist'):
                export_data[key] = value.tolist()
            elif isinstance(value, dict):
                export_data[key] = {}
                for k, v in value.items():
                    export_data[key][k] = v.tolist() if hasattr(v, 'tolist') else v
            else:
                export_data[key] = value
        
        with open(filename, 'w') as f:
            json.dump(export_data, f, indent=2, default=str)
        
        print(f"Results exported to {filename}")
        
    except Exception as e:
        print(f"Error during export: {str(e)}")

export_button.on_click(export_results)
display(export_button)

## Batch Analysis

Run pattern analysis on multiple units for comparison.

In [ ]:
# Simplified batch analysis
batch_units = widgets.SelectMultiple(
    options=unique_units[:20],  # Limit to first 20 for performance
    value=unique_units[:5],
    description='Select Units:',
    rows=8
)

batch_button = widgets.Button(
    description='Run Batch Analysis',
    button_style='primary'
)

def run_batch_analysis(button):
    selected_units = list(batch_units.value)
    
    if not selected_units:
        print("No units selected for batch analysis.")
        return
    
    print(f"Running batch analysis on {len(selected_units)} units...")
    
    # Get current parameters
    params = {
        'burst_max_isi': burst_max_isi.value,
        'burst_min_spikes': burst_min_spikes.value,
        'firing_rate_window': fr_window_size.value,
    }
    
    batch_results = {}
    for unit_id in selected_units:
        try:
            results = analyzer.run_pattern_analysis(unit_id, params)
            batch_results[unit_id] = results
        except Exception as e:
            batch_results[unit_id] = {'error': str(e)}
    
    # Print summary using utils function
    analyzer.print_batch_summary(batch_results)

batch_button.on_click(run_batch_analysis)

# Batch analysis interface
batch_box = widgets.VBox([
    widgets.HTML("<h3>Batch Analysis</h3>"),
    batch_units,
    batch_button
])

display(batch_box)

## Summary

This notebook provides a comprehensive interactive interface for spike pattern analysis. You can:

1. **Load Data**: Import spike data and electrode configuration
2. **Set Parameters**: Configure analysis parameters using interactive widgets
3. **Analyze Individual Units**: Run detailed pattern analysis on specific units
4. **Visualize Results**: Generate comprehensive plots and visualizations
5. **Export Results**: Save analysis results in JSON or CSV format
6. **Batch Analysis**: Compare multiple units simultaneously

The analysis includes:
- **Burst Detection**: Identify bursting patterns in spike trains
- **Firing Rate Analysis**: Calculate instantaneous and mean firing rates
- **ISI Analysis**: Inter-spike interval statistics and variability
- **Rhythmicity Analysis**: Detect periodic patterns using spectral analysis
- **Pattern Classification**: Automatically classify firing patterns

Use the controls above to explore your spike data interactively!